In [2]:
!pip install langchain_openai
!pip install langchain_chroma
!pip install langchain_groq
!pip install langchain_community
!pip install langchain_core
!pip install langchain_ray
!pip install chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 31.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.75
    Uninstalling langchain-core-0.3.75:
      Successfully uninstalled langchain-core-0.3.75
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [10]:
import pandas as pd
from langchain_chroma import Chroma
from langchain.docstore.document import Document
from langchain_community.vectorstores import chroma
from langchain_community.llms import openai
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
import ray
import os
import dask.dataframe as dd

In [4]:
import torch

print("CUDA available:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("CUDA device name:", torch.cuda.get_device_name(0))
    print("Current device index:", torch.cuda.current_device())

CUDA available: True
CUDA device count: 1
CUDA device name: NVIDIA A100-SXM4-40GB
Current device index: 0


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
dataframe = pd.read_csv("filtered_data2.csv")

In [12]:
len(dataframe)

37746

In [13]:
catgories = list(dataframe["Crop"].unique())

In [14]:
catgories

['Apple', 'Coconut', 'Paddy Dhan', 'Potato', 'Tomato']

In [15]:
!pip install langchain_huggingface

In [16]:
from langchain_huggingface import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings(model_name="multi-qa-MiniLM-L6-cos-v1",model_kwargs={"device": "cuda"})

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/383 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
import numpy as np

## Mapping the season names to its corresponding English names

In [18]:
season_map = {
    'JAYAD': 'Residual Crop',
    'RABI': 'Winter Crop',
    'KHARIF': 'Monsoon Crop',
}

In [19]:
dataframe["Season"].unique()

array(['JAYAD', nan, 'RABI', 'KHARIF', '0'], dtype=object)

In [20]:
dataframe['season_clean'] = dataframe['Season'].replace({np.nan: "", '0': ""})

In [21]:
dataframe['Season_English'] = dataframe['season_clean'].map(season_map)

In [22]:
from tqdm import tqdm

In [23]:
dataframe['Season_English'].unique()

array(['Residual Crop', nan, 'Winter Crop', 'Monsoon Crop'], dtype=object)

## Renaming Paddy Dhan to Paddy_Dhan

In [24]:
dataframe['Crop'] = dataframe['Crop'].replace('Paddy Dhan', 'Paddy_Dhan')

In [25]:
dataframe['Crop'].unique()

array(['Apple', 'Coconut', 'Paddy_Dhan', 'Potato', 'Tomato'], dtype=object)

## Getting the Unique categories

In [26]:
catgories = list(dataframe["Crop"].unique())

In [27]:
category_groups = {cat: dataframe[dataframe['Crop'] == cat] for cat in catgories}

In [28]:
len(dataframe["disease_name"].unique())

70

In [29]:
# Display all rows
pd.set_option('display.max_rows', None)

# Display full column width (not truncated)
pd.set_option('display.max_colwidth', None)

In [30]:
dataframe[["StateName","disease_name"]].value_counts()

StateName          disease_name                
ODISHA             Dhan stem borer                 1466
                   Dhan Blast                      1036
WEST BENGAL        Dhan stem borer                  905
UTTAR PRADESH      Dhan stem borer                  890
                   Dhan Leaf folder                 890
                   Dhan Blast                       830
TAMILNADU          Dhan Leaf folder                 814
ANDHRA PRADESH     Dhan stem borer                  777
WEST BENGAL        Dhan Blast                       762
ANDHRA PRADESH     Dhan Blast                       745
WEST BENGAL        Late blight                      599
JAMMU AND KASHMIR  scab                             580
ANDHRA PRADESH     Dhan Leaf folder                 578
UTTAR PRADESH      Late blight                      527
                   Dhan False smut                  491
TAMILNADU          Dhan stem borer                  475
ODISHA             Dhan Leaf folder                 438
ASSAM              Dhan stem borer                  397
TAMILNADU          Dhan Blast                       385
UTTAR PRADESH      Dhan Brown spot                  383
HARYANA            Dhan Blast                       344
PUNJAB             Dhan Leaf folder                 341
UTTAR PRADESH      Early blight                     328
                   Dhan Sheath blight               309
MADHYA PRADESH     Dhan Blast                       299
ODISHA             Dhan Sheath rot                  297
MADHYA PRADESH     Dhan stem borer                  292
ODISHA             Dhan Brown spot                  271
ASSAM              Late blight                      265
ANDHRA PRADESH     Dhan Sheath blight               253
CHHATTISGARH       Dhan stem borer                  253
PUNJAB             Dhan Blast                       245
ODISHA             Fruit borer                      241
WEST BENGAL        Dhan Brown spot                  241
TAMILNADU          Dhan Brown spot                  228
CHHATTISGARH       Dhan Blast                       227
ASSAM              Dhan Blast                       227
BIHAR              Dhan stem borer                  224
HIMACHAL PRADESH   scab                             221
ANDHRA PRADESH     Fruit borer                      220
KERALA             Bud rot                          217
                   Red palm weevil                  215
UTTAR PRADESH      Fruit borer                      205
ODISHA             Dhan False smut                  200
MADHYA PRADESH     Fruit borer                      196
TAMILNADU          Red palm weevil                  193
WEST BENGAL        Dhan Sheath rot                  193
MAHARASHTRA        Fruit borer                      192
                   Aphids                           191
TELANGANA          Dhan stem borer                  186
HARYANA            Dhan Leaf folder                 185
WEST BENGAL        Dhan Leaf folder                 184
                   Dhan Sheath blight               182
ODISHA             Dhan hispa                       182
ANDHRA PRADESH     Dhan False smut                  182
ODISHA             Red palm weevil                  181
MADHYA PRADESH     Late blight                      181
MAHARASHTRA        Dhan stem borer                  176
ODISHA             Dhan Sheath blight               176
UTTAR PRADESH      Dhan root weevil                 171
MAHARASHTRA        Leaf miner                       170
TAMILNADU          Fruit borer                      159
BIHAR              Late blight                      157
PUNJAB             Dhan Sheath blight               156
JAMMU AND KASHMIR  Rust                             155
PUNJAB             Dhan stem borer                  153
TAMILNADU          Dhan False smut                  148
UTTAR PRADESH      Dhan Bakanae                     148
KERALA             Rhinoceros beetle                143
HARYANA            Dhan stem borer                  142
WEST BENGAL        Dhan hispa   

In [ ]:
vc = dataframe[["StateName", "disease_name"]].value_counts().reset_index(name="count")

# Sort by StateName (alphabetical or numeric depending on dtype)
vc = vc.sort_values(by="StateName")

print(vc.to_string())

In [31]:
category_groups.keys()

dict_keys(['Apple', 'Coconut', 'Paddy_Dhan', 'Potato', 'Tomato'])

In [32]:
dataframe.columns

Index(['Unnamed: 0', 'DistrictName', 'StateName', 'Season', 'Month',
       'disease_name', 'QueryText', 'KccAns', 'Crop', 'word_count',
       'query_word_count', 'season_clean', 'Season_English'],
      dtype='object')

In [33]:
dataframe["StateName"].unique()

array(['JAMMU AND KASHMIR', 'HIMACHAL PRADESH', 'ODISHA',
       'ANDHRA PRADESH', 'UTTARAKHAND', 'RAJASTHAN', 'HARYANA',
       'MAHARASHTRA', 'ASSAM', 'WEST BENGAL', 'UTTAR PRADESH', 'BIHAR',
       'SIKKIM', 'MANIPUR', 'MADHYA PRADESH', 'DELHI', 'KERALA',
       'TAMILNADU', 'TRIPURA', 'KARNATAKA', 'GUJARAT', 'MEGHALAYA',
       'CHHATTISGARH', 'JHARKAND', 'TELANGANA', 'NAGALAND', 'PUDUCHERRY',
       'ARUNACHAL PRADESH', 'PUNJAB', 'MIZORAM', 'A AND N ISLANDS'],
      dtype=object)

## Creating the Vector Database

In [34]:
for cat, df_cat in tqdm(category_groups.items(),desc="Processing categories"):

    print(len(df_cat))
    docs = [
        Document(page_content=f"Crop:{cat}\nDisease:{row['disease_name']}\nStateName:{row['StateName']}\nSeason_English:{row['Season_English']}\nMonth:{row['Month']}\nQueryText:{row['QueryText']}\nKccAns:{row['KccAns']}",
                 metadata={"disease_name": row['disease_name'],"StateName":row["StateName"]})
        for _, row in df_cat.iterrows()
    ]

    chroma_collection = Chroma.from_documents(
        docs,
        embedding=embedding,
        persist_directory="./chroma_capstone_db_new",
        collection_name=cat
    )

Processing categories:   0%|          | 0/5 [00:00<?, ?it/s]

1837


Processing categories:  20%|██        | 1/5 [00:03<00:14,  3.52s/it]

2745


Processing categories:  40%|████      | 2/5 [00:06<00:09,  3.01s/it]

...

Processing categories:  60%|██████    | 3/5 [00:30<00:25, 12.55s/it]

4018


Processing categories:  80%|████████  | 4/5 [00:33<00:09,  9.12s/it]

4207


Processing categories: 100%|██████████| 5/5 [00:38<00:00,  7.68s/it]


In [ ]:
chroma_db = Chroma(
    persist_directory="./chroma_capstone_db_new",
    embedding_function=embedding,
    collection_name="Tomato"  # Specify which collection to load
)

In [ ]:
question = "give me the cure for tomato plant"
docs = chroma_db.max_marginal_relevance_search(question, k=3, fetch_k=6)

In [ ]:
!zip -r chroma_capstone_db_new_small2.zip chroma_capstone_db_new

  adding: chroma_capstone_db_new/ (stored 0%)
  adding: chroma_capstone_db_new/951f84ca-1e6a-41a6-87b8-276e788067c9/ (stored 0%)
  adding: chroma_capstone_db_new/951f84ca-1e6a-41a6-87b8-276e788067c9/data_level0.bin (deflated 12%)
  adding: chroma_capstone_db_new/951f84ca-1e6a-41a6-87b8-276e788067c9/length.bin (deflated 99%)
  adding: chroma_capstone_db_new/951f84ca-1e6a-41a6-87b8-276e788067c9/header.bin (deflated 58%)
  adding: chroma_capstone_db_new/951f84ca-1e6a-41a6-87b8-276e788067c9/link_lists.bin (deflated 82%)
  adding: chroma_capstone_db_new/951f84ca-1e6a-41a6-87b8-276e788067c9/index_metadata.pickle (deflated 45%)
  adding: chroma_capstone_db_new/chroma.sqlite3 (deflated 67%)
  adding: chroma_capstone_db_new/f4cf8ba5-89e5-4603-a8f8-5fb4734b3e6f/ (stored 0%)
  adding: chroma_capstone_db_new/f4cf8ba5-89e5-4603-a8f8-5fb4734b3e6f/data_level0.bin (deflated 12%)
  adding: chroma_capstone_db_new/f4cf8ba5-89e5-4603-a8f8-5fb4734b3e6f/length.bin (deflated 66%)
  adding: chroma_capstone_db

In [35]:
import shutil

# Zip the entire folder
shutil.make_archive("chroma_capstone_db", 'zip', "./chroma_capstone_db_new")

'/content/chroma_capstone_db.zip'